In [15]:
from scipy.io import loadmat
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import time
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.base import clone
from sklearn.model_selection import KFold


In [3]:
mnist = loadmat("mnist2/mnist-original.mat")
x = mnist["data"].T
y = mnist["label"][0]
split = 60000
train_x, train_y, test_x, test_y = x[:split], y[:split], x[split:], y[split:]
print(train_x.shape, test_x.shape)
X_01 = x[(y == 0) | (y == 1)]
y_01 = y[(y == 0) | (y == 1)]

(60000, 784) (10000, 784)


In [4]:
train_x01 = train_x[(train_y == 0) | (train_y == 1)].copy()
train_y01 = train_y[(train_y == 0) | (train_y == 1)].copy()
test_x01 = test_x[(test_y == 0) | (test_y == 1)].copy()
test_y01 = test_y[(test_y == 0) | (test_y == 1)].copy()

In [5]:
print(train_x01.shape, test_x01.shape)

(12665, 784) (2115, 784)


In [6]:
def cross_validation(classifier, X, y, k, shuffle=False):
    kf = KFold(n_splits=k, shuffle=shuffle)
    trained_classifier = []
    accuracy_list = []
    for train_index, test_index in kf.split(X):
        X_train_kfold, X_test_kfold = X[train_index], X[test_index]
        y_train_kfold, y_test_kfold = y[train_index], y[test_index]
        classifier_fold = clone(classifier)
        classifier_fold.fit(X_train_kfold, y_train_kfold)
        prediction = classifier_fold.predict(X_test_kfold)
        accuracy = accuracy_score(y_test_kfold, prediction)
        trained_classifier.append(classifier_fold)
        accuracy_list.append(accuracy)
    return trained_classifier, accuracy_list

In [7]:
def check_classifier(classifier, X_train, y_train, X_val, y_val):
    trained_classifier, accuracy_list = cross_validation(classifier, X_train, y_train, 5)
    accuracy = 0.0
    for i in range(len(accuracy_list)):
        accuracy += accuracy_score(y_val, trained_classifier[i].predict(X_val))
    
    mean_accuracy = accuracy / len(accuracy_list)
    print(accuracy_list)
    best_classifier = trained_classifier[np.argmax(accuracy_list)]
    return best_classifier, mean_accuracy

In [17]:
#test linear boosting vs logistic regression

[0.9984208448480063, 0.9968416896960126, 0.9988156336360048, 0.9976312672720095, 0.9988156336360048]
0.9989598108747046
accuracy:  0.9990543735224586


In [17]:
logreg_clf = LogisticRegression(solver="saga", tol=0.01)
start = time.time()
logreg_clf.fit(train_x01, train_y01)
print("time: ", time.time() - start)
prediction = logreg_clf.predict(test_x01)

print("score: ", logreg_clf.score(test_x01, test_y01))

time:  5.09523606300354
score:  0.9990543735224586
accuracy:  0.9990543735224586


In [18]:
lingb_clf = XGBClassifier(booster='gblinear')
start = time.time()
lingb_clf.fit(train_x01, train_y01)
print("time: ", time.time() - start)
prediction = lingb_clf.predict(test_x01)
print("score: ", lingb_clf.score(test_x01, test_y01))

time:  2.229208469390869
score:  0.9995271867612293


In [22]:
# linreg_clf = LinearRegression()
# start = time.time()
# linreg_clf.fit(train_x01, train_y01)
# print("time: ", time.time() - start)
# prediction = linreg_clf.predict(test_x01)
# print("accuracy: ", linreg_clf.score(test_x01, test_y01))

time:  1.8840653896331787
accuracy:  -482244559518.77014
